# Estimación probabilística de la dosis de exposición oral


## Estimación de la concentración de la sustancia de interés en los alimentos

- *mean* corresponde al valor de la media de la concentración de la sustancia de interés en la solución a considerar.
- *stdev* es la desviación estándar de la concentración de la sustancia de interés en la solución utilizada.
- *CV* es el coeficiente de variación es la desviación estándar relativa  de la concentración de la sustancia de interés en la solución utilizada.
- *dosis* es la cantidad de mg de ingrediente activo por hectarea (mg *i.a*/Ha) a ser utilizada durante la aplicación del producto químico de uso agrícola de que se trate.
- *volumen de aspersión* es el volumen por hectarea (L/Ha) de solución o dispersión de la sustancia de interés a ser utilizada.


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.stats import lognorm
import ipywidgets as widgets
from IPython.display import display, clear_output

%matplotlib inline

alimean = {'tomate': 0.2, 'cítricos': 0.09, 'maíz': 0.01, 'papa': 0.01, 'soya': 0.01}
alicv = {'tomate': 1.051, 'cítricos': 1.122, 'maíz': 1.1, 'papa': 1.051, 'soya': 1.1}

food_sliders = [widgets.FloatSlider(value=mean, min=0.01, max=0.5, step=0.01, description=f'cv {food}', continuous_update=False) for food in alimean]

prbsrl_slider = widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description='prbsrl', continuous_update=False)


pesokde = np.genfromtxt('../../../BD/pesokde.csv', delimiter=',', skip_header=1)
tsnglm = [np.genfromtxt(f'tasaing/csv/tasa_ing_{alim}.csv', delimiter=',', skip_header=1) for food in alimean]

dtcnclt = []

def lgnrmcnc(mean, stdev):
    phi = (stdev ** 2 + mean ** 2) ** 0.5
    mu = np.log(mean ** 2 / phi)
    sigma = (np.log(phi ** 2 / mean ** 2)) ** 0.5
    data = np.random.lognormal(mu, sigma , 1000)
    return data, mu, sigma

def plot_lognormal(ax, data, mean, sigma, food_name, cv, prbsrl):
    ax.hist(data, bins=30, density=True, alpha=0.5, color="b")
    xmin, xmax = ax.get_xlim()
    x = np.linspace(xmin, xmax, 1000)
    p = lognorm.pdf(x=x, scale=mean, s=sigma)
    ax.plot(x, p, 'k', linewidth=2)
    ax.set_title(f'Distribución lognormal para {food_name} \n (CV: {cv}, prbsrl: {prbsrl})')
    ax.set_ylabel('Densidad de probabilidad')
    ax.set_xlabel('mg/kg', y=0.05)

def hstgrm(ax, data, bins):
    ax.hist(data, density=True, bins=bins)
    ax.set_ylabel('Densidad de probabilidad')
    ax.set_xlabel('mg/kg·día');
    ax.axvline(x = np.percentile(data, 50), linewidth=1, color='r', linestyle=':')
    ax.axvline(x = np.percentile(data, 2.5), linewidth=1, color='r', linestyle='--')
    ax.axvline(x = np.percentile(data, 97.5), linewidth=1, color='r', linestyle='dashed')
    ax.text((np.percentile(data, 97.5)+0.05*iqr(data)), (len(data)/(sum(data)+0.5)), 'P97.5 = '+np.format_float_scientific(np.percentile(data, 97.5), precision=2), rotation=90, color='red', verticalalignment='center')
    ax.set_xlim(0, np.percentile(data, 97.5)+iqr(data))
    ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0), useMathText=True)

def bxplt(ax, data):
    ax.boxplot(np.array(data)[np.array(data) < np.percentile(data, 97.5)+iqr(data)], vert=False, patch_artist=True)
    ax.set_xlabel('mg/kg·día')
    ax.set_yticks([1], [''])
    ax.axvline(x = np.quantile(data, 0.975) , linewidth=1, color='r', linestyle='--')
    ax.text((np.percentile(data, 97.5)+0.05*iqr(data)), 1.125, 'P97.5 = '+np.format_float_scientific(np.percentile(data, 97.5), precision=2), color='red', rotation=90)
    ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0), useMathText=True)

def montecarlo(prbsrl, N):
    exporal = []
    for elem in range(len(dtcnclt)):
        exporalim = []
        for i in range(N):
            random_concalim = np.random.choice(dtcnclt[elem])
            random_tasaing = np.random.choice(tsnglm[elem])
            random_peso = np.random.choice(pesokde)
            exporal_i = ((random_concalim * (random_tasaing / 1000) * prbsrl) / random_peso)
            exporalim.append(exporal_i)
        exporal.append(exporalim)
    return exporal

bins = 1500
lstlm = ['tomate', 'cítricos', 'maíz', 'papa', 'soya']  # List to be used in hstgrm function for file naming

output_widget = widgets.Output()

def update_plot(change):
    with output_widget:
        clear_output(wait=True)
        
        # Set up the grid
        fig = plt.figure(figsize=(20, 15))  # Adjust size to your needs
        gs = gridspec.GridSpec(3, 5, figure=fig)
        
        # Lognormal plots row
        dtcnclt[:] = [lgnrmcnc(slider.value, slider.value * alicv.get(slider.description.split(" ")[-1], 0.0))[0] for slider in food_sliders]
        prbsrl = prbsrl_slider.value
        for index, data in enumerate(dtcnclt):
            ax = fig.add_subplot(gs[0, index])
            _, mu, sigma = lgnrmcnc(food_sliders[index].value, food_sliders[index].value * alicv.get(food_sliders[index].description.split(" ")[-1], 0.0))
            plot_lognormal(ax, data, np.exp(mu), sigma, lstlm[index], food_sliders[index].value, prbsrl)
        
        # Histogram plots row
        xprl = montecarlo(prbsrl, 5000)
        for index, data in enumerate(xprl):
            ax = fig.add_subplot(gs[1, index])
            hstgrm(ax, data, bins)
            ax.set_title(f'Distribución de la dosis de exposición para \n consumo de {lstlm[index]} (CV: {food_sliders[index].value}, prbsrl: {prbsrl})')

        
        # Boxplots row
        for index, data in enumerate(xprl):
            ax = fig.add_subplot(gs[2, index])
            bxplt(ax, data)
        
        plt.tight_layout()  # Adjusts spacing between subplots for better readability
#         plt.savefig('summary_oral.png', bbox_inches='tight', dpi=300)
        plt.show()

prbsrl_slider.observe(update_plot, names='value')
for food_slider in food_sliders:
    food_slider.observe(update_plot, names='value')

display(widgets.VBox([widgets.HBox(food_sliders), prbsrl_slider, output_widget]))

# Initially populate the output widget
update_plot(None)